In [202]:
%load_ext autoreload
%autoreload 2
# Enable imports form top-level of project (edit top_level_path accordingly)
import os
import sys
top_level_path = os.path.abspath(os.path.join('..'))
if top_level_path not in sys.path:
	sys.path.append(top_level_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [203]:

from longestpath import gen_average_degree_directed, gen_planted_path, StandardGraph, complete_graph, gen_num_edges, gen_num_edges_no_loops
from longestpath import qubo
from longestpath import brute
import multiprocessing
import time

In [204]:
n = 15
d = 3
graph = gen_num_edges_no_loops(n, round(n * d))

def pretty_result(result):
	info = {k:v for k,v in result.items() if type(v) in [int, float, str]}
	if "path" in result:
		info["length"] = len(result['path'])-1
		format_str = "{:>3},"*len(result['path'])
		return str(info) + "\n" + format_str.format(*list(map(str, result['path'])))
	else:
		return str(info)

brute_result = brute.solve(multiprocessing.Queue(), graph, "BRANCH_N_BOUND")
print(pretty_result(brute_result))
target = len(brute_result['path'])-1
print("Target =", target)

solver = qubo.QUBOSolver(graph)
beginTime = time.time()

min_beta, max_beta = solver.get_default_beta_range()
e, sampleset = solver.solve_for_sample(num_reads=100, num_sweeps=100)#, beta_range=(0.1, 0.5*max_beta))
while(-e < target):
	e, sampleset = solver.solve_for_sample(num_reads=100, num_sweeps=100, initial_states= sampleset)#, beta_range=(0.1, 0.5*max_beta))

endTime = time.time()
print("Time to solve = ", endTime - beginTime)
print(-e)

{'run_time': 0.0, 'length': 7}
  0,  2,  6,  7,  9, 11, 13, 14,
Target = 7
Time to solve =  17.830904960632324
7.0
